<a href="https://colab.research.google.com/github/daniel-lara-ec/Proy-Actuarial2023A/blob/main/Ajuste/Actuarial_TG1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Módulos

In [ ]:
import pandas as pd
import requests
import plotly.graph_objects as go
import numpy as np
import kaleido

In [ ]:
url = "https://pruebas.alephsub0.org/EDG_2021_CSV.csv"

In [ ]:
storage_options = {'User-Agent': 'Mozilla/5.0'}
df = pd.read_csv(url,sep=';',storage_options=storage_options)

<ipython-input-94-5a8d52acd6d0>:2: DtypeWarning:

Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.



In [ ]:
df.head()

,Numeracion,prov_insc,cant_insc,parr_insc,anio_insc,mes_insc,dia_insc,fecha_insc,nac_fall,cod_pais,...,area_res,est_civil,niv_inst,residente,causa,lc1,causa103,causa80,causa67A,causa67B
0,7586,Guayas,Milagro,Milagro,2021,Octubre,04,2021-10-04,Ecuatoriana,Ecuador,...,Urbano,Soltero (a),Educación Básica,No residente,O00 Embarazo ectópico,"54 Embarazo, parto y puerperio",088 Embarazo terminado en aborto,066 Embarazo terminado en aborto,No aplica,No aplica
1,9158,Guayas,Guayaquil,Tarqui,2021,Abril,03,2021-04-03,Ecuatoriana,Ecuador,...,Urbano,Unido (a),Secundaria,No residente,O00 Embarazo ectópico,"54 Embarazo, parto y puerperio",088 Embarazo terminado en aborto,066 Embarazo terminado en aborto,No aplica,No aplica
2,61037,Los Ríos,Quevedo,San Camilo,2021,Febrero,19,2021-02-19,Ecuatoriana,Ecuador,...,Urbano,Unido (a),Secundaria,Residente,O00 Embarazo ectópico,"54 Embarazo, parto y puerperio",088 Embarazo terminado en aborto,066 Embarazo terminado en aborto,No aplica,No aplica
3,67063,Pichincha,Quito,Calderón (Carapungo),2021,Octubre,20,2021-10-20,Ecuatoriana,Ecuador,...,Rural,Unido (a),Secundaria,Residente,O00 Embarazo ectópico,"54 Embarazo, parto y puerperio",088 Embarazo terminado en aborto,066 Embarazo terminado en aborto,No aplica,No aplica
4,102146,Esmeraldas,Ríoverde,Rocafuerte,2021,Noviembre,29,2021-11-29,Ecuatoriana,Ecuador,...,Urbano,Unido (a),Superior universitario,No residente,"O08 Complicaciones consecutivas al aborto, a...","54 Embarazo, parto y puerperio",091 Resto de embarazo parto y puerperio,066 Embarazo terminado en aborto,No aplica,No aplica


In [ ]:
provincias_oriente =  ['Pastaza','Sucumbíos', 'Napo','Orellana', 'Morona Santiago', 'Zamora Chinchipe']
df_oriente = df[df["prov_res"].isin(provincias_oriente)]
df_oriente['edad'] = df_oriente['edad'].astype(str)

<ipython-input-96-52fca40a305a>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df2 = pd.DataFrame(df_oriente.groupby(['edad','sexo']).size(),columns=['defunciones'])
df2 = df2.drop(['Sin información'])
df2.head()

defunciones
edad sexo               
0    Hombre            1
     Mujer             8
1    Hombre           28
     Mujer            42
10   Hombre            7

In [ ]:
#pivot sexo to columns
df2 = df2.reset_index().pivot(index='edad', columns='sexo', values='defunciones')
df2['edad'] = df2.index.astype(int)
# drop index
df2 = df2.reset_index(drop=True)
df2 = df2.sort_values('edad')
df2 = df2.reset_index(drop=True)
# change columns to int replace nan with 0
df2['Hombre'] = df2['Hombre'].fillna(0).astype(int)
df2['Mujer'] = df2['Mujer'].fillna(0).astype(int)
# rename edad to Edad
df2 = df2.rename(columns={'edad':'Edad'})

In [ ]:
df2.head()

defunciones
edad sexo               
0    Hombre            1
     Mujer             8
1    Hombre           28
     Mujer            42
10   Hombre            7

In [ ]:
df2H = df2[["Edad","Hombre"]]
df2M = df2[["Edad","Mujer"]]

In [ ]:
lxh = []
t = df2H["Hombre"].sum()
for i in range(0,df2H["Hombre"].size):
    lxh.append(t)
    t=t-df2H["Hombre"][i]

lxm = []
t = df2M["Mujer"].sum()
for i in range(0,df2M["Mujer"].size):
    lxm.append(t)
    t=t-df2M["Mujer"][i]

In [ ]:
df2H["lx"] = lxh
df2H["qx"] = df2H["Hombre"]/df2H["lx"]
df2M["lx"] = lxm
df2M["qx"] = df2M["Mujer"]/df2M["lx"]

In [ ]:
df2H.head()

sexo,Edad,Hombre,lx,qx
0,0,1,2511,0.000398
1,1,28,2510,0.011155
2,2,26,2482,0.010475
3,3,21,2456,0.008550
4,4,14,2435,0.005749


In [ ]:
df2M.head()

sexo,Edad,Mujer,lx,qx
0,0,8,1642,0.004872
1,1,42,1634,0.025704
2,2,17,1592,0.010678
3,3,12,1575,0.007619
4,4,13,1563,0.008317


In [ ]:
df2["Totaldx"]= df2["Hombre"]+df2["Mujer"]
lxt = []
t = df2["Totaldx"].sum()
for i in range(0,df2["Totaldx"].size):
    lxt.append(t)
    t=t-df2["Totaldx"][i]

df2["lx"] = lxt
df2["qx"] = df2["Totaldx"]/df2["lx"]
df2.head()

sexo,Hombre,Mujer,Edad,Totaldx,lx,qx
0,1,8,0,9,4153,0.002167
1,28,42,1,70,4144,0.016892
2,26,17,2,43,4074,0.010555
3,21,12,3,33,4031,0.008187
4,14,13,4,27,3998,0.006753


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df2H["Edad"], y=df2H["qx"],mode='lines',name='Hombre'))
fig.add_trace(go.Scatter(x=df2M["Edad"], y=df2M['qx'],mode='lines',name='Mujer'))
fig.add_trace(go.Scatter(x=df2["Edad"], y=df2['qx'],mode='lines',name='Total'))
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df2H["Edad"], y=np.log(df2H["qx"]),mode='lines',name='Hombre'))
fig.add_trace(go.Scatter(x=df2M["Edad"], y=np.log(df2M['qx']),mode='lines',name='Mujer'))
fig.add_trace(go.Scatter(x=df2["Edad"], y=np.log(df2['qx']),mode='lines',name='Total'))
fig.show()

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning:

divide by zero encountered in log



## Datos tabulados

> Indented block



In [ ]:
url2 = "https://github.com/daniel-lara-ec/Proy-Actuarial2023A/raw/main/defunciones.xlsx"

In [ ]:
df = pd.read_excel(url2)
df.head()

,Hombre,Mujer,Edad
0,1,6,0
1,21,33,1
2,19,14,2
3,15,11,3
4,10,12,4


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Edad'], y=df['Hombre'],mode='lines',name='Hombre'))
fig.add_trace(go.Scatter(x=df['Edad'], y=df['Mujer'],mode='lines',name='Mujer'))
fig.add_trace(go.Scatter(x=df['Edad'], y=df['Mujer']+df['Hombre'],mode='lines',name='Total'))
fig.show()


# **Método de Wittaker-Henderson** (Suavizamiento)

La fórmula de Wittaker Henderson implica minimizar la cantidad

$$
\mathcal{J}=\sum_{n=0}^{N-1} w_n \left|y_n - x_n\right|^{2}+ \lambda \sum_{n=0}^{N-1} \left|\nabla^{s} x_n\right|^{2},
$$
donde:
* $w_n$ es un coeficiente de ponderación,
* $y_n$ son los valores que se desean suavizar,
* $x_n$ son los valores suavizados,
* $\lambda$ es un parámetro que regula la importancia que el usuario asigna a la suavidad, frente a la proximidad a los datos observados, y
* $\nabla^{s} x_n$ es operador de diferencia hacia atras $\nabla x_n = x_n - x_{n-1}$ aplicado $s$ veces.

Respecto al parámetro $\lambda$, mientras más pequeño sea su valor, las tasas suavizadas serás más cercanas a las $y_n$ previamente calculadas. En un caso extremo, si $\lambda=0$, el mínimo se verifica cuando los valores de $x_n$ coinciden con los de $y_n$, mientras que en el otro caso extremo, si $\lambda \to +\infty$, entonces los valores suavizados seguiran el comportamiento de una linea recta.


En su implementación para las tablas de mortalidad en Estados Unidos de 1961-1971, Greenville consideró:
$$
w_x=\frac{E_x}{q_{x(obs)}(1-q_{x(obs)})} \quad\text{y }\quad s=2.
$$

<br/>
<br/>
Podemos reescribir el problema de minimización en una forma más compacta de la la siguiente manera:
$$
\mathcal{J}=(y-x)^T W (y-x)+\lambda x^T(D^T D)x,
$$
donde $W$ es la matriz diagonal de pesos, $W=diag(w_1, \cdots , w_{N-1})$.La matriz $D^T D$ es escencialmente el operador $\nabla^{2s} $, después de ignorar las primeras $s$ y las últimas $s$ filas. Por ejemplo, para $N=7$ y $s=2$:

$$
D^T D=\left[
\begin{array}{r}
1  & -2 & 1 & 0 & 0 & 0 & 0\\
-2 & 5 & -4 & 1 & 0 & 0 & 0\\
1  & -4 & 6 & 4 & 1 & 0 & 0\\
0  & 1 & -4 & 6 & 4 & 1 & 0\\
0  & 0 & 1 & 4 & 6 & 4 & 1\\
1  & -2 & 1 & 0 & 0 & 0 & 0\\
1  & -2 & 1 & 0 & 0 & 0 & 0
\end{array}
\right]
\\[5mm]
$$

Luego, la solución optima se obtiene igualando el gradiente respecto a $x$ a cero, es decir:
$$
\frac{\partial \mathcal{J}}{\partial x}=-2W(y-x)+2 \lambda (D^T D)x=0,
$$
lo que implica que:
$$
(W+\lambda D^T D)x=Wy.
$$
De esta última ecuación buscamos los valores suavizados $x$, para ello debemos invertir la $N\times N$ matriz $W+\lambda D^T D$; esto es posible debido a que esta matriz es sparse y tiene ancho de banda $2s+1$ por que los métodos usuales de inversión funcionan adecuadamente.

In [ ]:
import math
import pandas as pd
import requests
import plotly.graph_objects as go
import numpy as np

In [ ]:
def coef(s):
  coefi=np.zeros(s+1)
  for k in range(s+1):
    coefi[k]=(-1)**(k)*math.factorial(s)/(math.factorial(k)*(math.factorial(s-k)))
  return coefi

In [ ]:
coef(3)

array([ 1., -3.,  3., -1.])

In [ ]:
#Función para la matriz D^T *D según los valores de N y s.
def mat_DTD(N,s):
  D=np.zeros([N-s, N])
  coefi=coef(s)
  for i in range(N-s):
    for j in range(s+1):
      D[i,j+i]=coefi[j]

  F=np.matmul(D.transpose(),D)
  return F

In [ ]:
mat_DTD(7,2)

array([[ 1., -2.,  1.,  0.,  0.,  0.,  0.],
       [-2.,  5., -4.,  1.,  0.,  0.,  0.],
       [ 1., -4.,  6., -4.,  1.,  0.,  0.],
       [ 0.,  1., -4.,  6., -4.,  1.,  0.],
       [ 0.,  0.,  1., -4.,  6., -4.,  1.],
       [ 0.,  0.,  0.,  1., -4.,  5., -2.],
       [ 0.,  0.,  0.,  0.,  1., -2.,  1.]])

#Datos

In [ ]:
df2=pd.read_excel('basefinalfinal2.xlsx')
df2.head()

,Unnamed: 0,Hombre,Mujer,Edad,Totaldx,lxH,lxM,qxH,qxM,lxT,qxT,qxHo,qxMo,qxTo
0,0,1,8,0,9,9500,9182,0.000105,0.000871,18682,0.000482,0.000105,0.000871,0.000482
1,1,28,42,1,70,8907,8466,0.003144,0.004961,17373,0.004029,0.003144,0.004961,0.004029
2,2,26,17,2,43,10776,10336,0.002413,0.001645,21112,0.002037,0.002413,0.001645,0.002037
3,3,21,12,3,33,12485,12040,0.001682,0.000997,24525,0.001346,0.001682,0.000997,0.001346
4,4,14,13,4,27,12008,11496,0.001166,0.001131,23504,0.001149,0.001166,0.001131,0.001149


#Suavizamiento Hombres

In [ ]:
ch=pd.read_excel('Cohortes.xlsx',sheet_name='CohorteHombres')

In [ ]:
#Matriz WH:
dh_x=df2[0:99]["Hombre"]
E_x=ch[0:99]["2021_final"]+0.5*dh_x
wh_x=E_x/(df2[0:99]["qxHo"]*(1-df2[0:99]["qxHo"]))

WH=np.diag(wh_x)
WH

array([[9.02642515e+07, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.84678295e+06, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 4.48244063e+06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.00122927e+03, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.84121212e+02, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 3.37254902e+02]])

In [ ]:
DTDh1=mat_DTD(df2[0:99]["qxH"].size,2)
lamb=1500
q_suav1=np.matmul(np.linalg.inv(WH+lamb*DTDh1),WH@df2[0:99]["qxH"])

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df2["Edad"], y=q_suav1,mode='lines',name='qxH suav B'))
fig.add_trace(go.Scatter(x=df2["Edad"], y=df2['qxHo'],mode='lines',name='qxH'))
fig.show()

In [ ]:
w_n=np.ones(df2[0:99]["qxH"].size)
W=np.diag(w_n)
DTDh2=mat_DTD(df2[0:99]["qxH"].size,2)
lamb=0.9
q_suav2=np.matmul(np.linalg.inv(W+lamb*DTDh2),W@df2[0:99]["qxH"])

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df2["Edad"], y=q_suav2,mode='lines',name='qxH suav A'))
fig.add_trace(go.Scatter(x=df2["Edad"], y=q_suav1,mode='lines',name='qxH suav B'))
fig.add_trace(go.Scatter(x=df2["Edad"], y=df2['qxHo'],mode='lines',name='qxH'))
fig.show()

#Suavizamiento Mujeres

In [ ]:
cm=pd.read_excel('Cohortes.xlsx',sheet_name='CohorteMujeres')

In [ ]:
#Matriz WM:
dm_x=df2[0:99]["Mujer"]
E_xm=cm[0:99]["2021_final"]+0.5*dm_x
wm_x=E_xm/(df2[0:99]["qxMo"]*(1-df2[0:99]["qxMo"]))

WM=np.diag(wm_x)
WM

array([[1.05524255e+07, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.71926603e+06, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 6.29981754e+06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.19700181e+03, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 9.62666667e+01, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.30605769e+02]])

In [ ]:
DTDm1=mat_DTD(df2[0:99]["qxM"].size,2)
lamb=1500
q_suav3=np.matmul(np.linalg.inv(WM+lamb*DTDm1),WM@df2[0:99]["qxM"])

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df2["Edad"], y=q_suav3,mode='lines',name='qxM suav B'))
fig.add_trace(go.Scatter(x=df2["Edad"], y=df2['qxMo'],mode='lines',name='qxM'))
fig.show()

In [ ]:
DTDm2=mat_DTD(df2[0:99]["qxM"].size,2)
lamb=0.9
q_suav4=np.matmul(np.linalg.inv(W+lamb*DTDm2),W@df2[0:99]["qxM"])

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df2["Edad"], y=q_suav4,mode='lines',name='qxM suav A'))
fig.add_trace(go.Scatter(x=df2["Edad"], y=q_suav3,mode='lines',name='qxM suav B'))
fig.add_trace(go.Scatter(x=df2["Edad"], y=df2['qxMo'],mode='lines',name='qxM'))
fig.show()

In [ ]:
DTDt=mat_DTD(df2[0:99]["qxT"].size,2)
lamb=0.9
q_suav5=np.matmul(np.linalg.inv(W+lamb*DTDt),W@df2[0:99]["qxT"])

# Comparación Suavizamientos

In [ ]:
df3=pd.read_excel('TablaSuavizada.xlsx')
df3.head()

,Unnamed: 0.1,Unnamed: 0,qxH_c,qxM_c,qxT_c,qxH_cc,qxM_cc,qxT_cc
0,0,0,0.000575,0.002079,0.001393,0.000700,0.002211,0.001519
1,1,1,0.000928,0.001923,0.001581,0.001085,0.002006,0.001677
2,2,2,0.002337,0.002011,0.002227,0.002349,0.002088,0.002265
3,3,3,0.001679,0.001228,0.001466,0.001690,0.001233,0.001472
4,4,4,0.001239,0.001014,0.001136,0.001244,0.001015,0.001138


In [ ]:
df3['qxH_sA']=q_suav2
df3['qxH_sB']=q_suav1
df3['qxM_sA']=q_suav4
df3['qxM_sB']=q_suav3
df3['qxT_sA']=q_suav5

In [ ]:
df3.to_excel('S2.xlsx')

In [ ]:
df3.rename(columns={'Unnamed: 0.1':'Edad'},inplace=True)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df2["Edad"], y=df2['qxMo'],mode='lines',name='qxM obs'))
fig.add_trace(go.Scatter(x=df2["Edad"], y=df2['qxHo'],mode='lines',name='qxH obs'))
fig.show()

# Conclusiones

1. Podemos observar que las probabilidades de muerte $q_x$ obtenidas luego de realizar el proceso intercensal no se ajustan a lo esperado; es decir, la probabilidad de muerte no necesariamente crece a medida que la edad aumenta. Estas tampoco verifican que al llegar al infinito actuarial ($w=98$), $q_x=1$. Por lo tanto, se realizo un ajusto de datos previo al suavizamiento mediante una media movil.

2. Luego de realizar el ajuste mediante la media movil, vemos que las probabilades $q_x$ ahora siguen un comportamiento más acorde al esperado; sin embargo, estas todavía presentan algunos "picos" los cuales serán suavizados mediante el método de Whittaker-Henderson (W-H).

3. Notamos que los métodos W-H tipo A ($W=Id$) y tipo B ($W\neq Id $) producen resultados similares. No obstante, ambos métodos se comportan de manera opuesta. Por un lado,en el tipo A, al considerar $W=Id$, obtenemos una suavidad moderada con $\lambda=0.9\leq 1$.

  Por otro lado, en el método tipo B, dado que consideramos las ponderaciones

  $$
w_x=\frac{E_x}{q_{x(obs)}(1-q_{x(obs)})},
$$
  el método tiende a arrojar valores más cercanos a los observados. Por lo que para tener un suavizamiento adecuado se deben considerar valores de $\lambda$ altos. En nuestro caso, consideramos $\lambda=1000$.

4. Ambos métodos de suavizamiento no presentan problemas en las primeras edades ($< 80$), pues allí las probabilidades calculadas no varían mucho.
